# Palyginimas su pirminiu šaltiniu



In [ ]:
import json
import os
from zipfile import ZipFile
from typing import Optional

import pandas as pd
import plotly.express as px
import requests

working_directory = f'{os.getcwd()}/../data/saltiniai/vintra/'
stops_lt_directory = f'{os.getcwd()}/../data/saltiniai/stops-lt/'
gtfs_files_directory = os.path.join(working_directory, 'gtfs')
stops_lt_gtfs_directory = os.path.join(stops_lt_directory, 'gtfs')
netex_directory = os.path.join(working_directory, 'netex')

mapbox_access_token = open("../.mapbox_token").read()
px.set_mapbox_access_token(mapbox_access_token)

lithuania_center = {'lat': 55.169438, 'lon': 23.881275}

municipalities_gtfs_file_mapping = pd.read_csv(
    os.path.join(working_directory, 'vintra-gtfs-file-mapping.csv'),
    na_filter=False)
vintra_netex_mapping = pd.read_csv(os.path.join(working_directory, 'vintra-netex-file-mapping.csv'), na_filter=False)
stops_lt_gtfs_mapping = pd.read_csv(os.path.join(stops_lt_directory, 'stops-lt-file-mapping.csv'), na_filter=False)

with open('../data/geojson/municipalities.geojson', 'r') as municipalities_geojson_file:
    municipalities_geojson = json.load(municipalities_geojson_file)

In [ ]:
gtfs_files_by_specification = [
    'agency.txt',
    'stops.txt',
    'routes.txt',
    'trips.txt',
    'stop_times.txt',
    'routes.txt',
    'trips.txt',
    'stop_times.txt',
    'calendar.txt',
    'calendar_dates.txt',
    'shapes.txt',
    'feed_info.txt',
    'fare_rules.txt',
    'fare_attributes.txt',
    'translations.txt',
    'frequencies.txt',
    'transfers.txt',
    'pathways.txt',
    'levels.txt',
    'attributions.txt',
]

stops_lt_file_stats = stops_lt_gtfs_mapping[stops_lt_gtfs_mapping['Failas'] != '']


def calculate_rows_in_gtfs_file(full_file: str, gtfs_file: str):
    with ZipFile(full_file) as gtfs_zip:
        gtfs_zip_files = gtfs_zip.namelist()
        if gtfs_file not in gtfs_zip_files:
            return -1

        df = pd.read_csv(gtfs_zip.open(gtfs_file))

        return len(df)


for gtfs_file in gtfs_files_by_specification:
    stops_lt_file_stats[gtfs_file] = stops_lt_file_stats['Failas'].apply(
        lambda f: calculate_rows_in_gtfs_file(os.path.join(stops_lt_gtfs_directory, f), gtfs_file),
    )

Dalis Lietuvos savivaldybių naudoja Stops.lt sistemą, kurios duomenys GTFS formatu yra viešai prieinami bei automatiniu būdu importuojami į VINTRA.

Toliau pateikiama lentelė vaizduoja kiekvienos savivaldybės naudojančios Stops.lt sistemą GTFS rinkinio failus bei eilučių skaičių juose.

```{table} Stops.lt sistemą naudojančių savivaldybių GTFS duomenų rinkinio failai ir juose esančių eilučių skaičius
:name: municipalities-stops-lt-stats
| Savivaldybė                  | agency.txt | stops.txt | routes.txt | trips.txt | stop\_times.txt | calendar.txt | calendar\_dates.txt | shapes.txt | fare\_rules.txt | fare\_attributes.txt | attributions.txt |
|------------------------------|------------|-----------|------------|-----------|-----------------|--------------|---------------------|------------|-----------------|----------------------|------------------|
| Alytaus miesto savivaldybė   | 1          | 196       | 19         | 955       | 21483           | 26           | 200                 | 12663      | 0               | 0                    | 0                |
| Druskininkų savivaldybė      | 1          | 149       | 27         | 709       | 9043            | 18           | 216                 | 9793       | 0               | 0                    | 0                |
| Kauno miesto savivaldybė     | 1          | 885       | 63         | 7320      | 190006          | 99           | 1287                | 84486      | 126             | 4                    | 0                |
| Klaipėdos miesto savivaldybė | 6          | 635       | 55         | 6976      | 164451          | 107          | 1365                | 24253      | 0               | 0                    | 0                |
| Panevėžio miesto savivaldybė | 1          | 241       | 20         | 1687      | 28846           | 29           | 37                  | 20378      | 0               | 0                    | 0                |
| Šiaulių miesto savivaldybė   | 1          | 517       | 56         | 2922      | 59738           | 75           | 150                 | 36184      | 0               | 0                    | 0                |
| Vilniaus miesto savivaldybė  | 1          | 1432      | 99         | 25290     | 508956          | 271          | 3510                | 110484     | 0               | 0                    | 99               |
```

In [ ]:
vintra_and_stops_lt_files = stops_lt_gtfs_mapping[stops_lt_gtfs_mapping['Failas'] != ''].merge(
    municipalities_gtfs_file_mapping, suffixes=('_stops', '_vintra'), on='Savivaldybe')


for gtfs_file in gtfs_files_by_specification:
    vintra_and_stops_lt_files[gtfs_file] = vintra_and_stops_lt_files['Failas_vintra'].apply(
        lambda f: calculate_rows_in_gtfs_file(os.path.join(gtfs_files_directory, f), gtfs_file),
    )

vintra_and_stops_lt_files.to_excel('vintra-files.xlsx')

Kai šių savivaldybių GTFS duomenų rinkinio failai importuojami į Stops.lt turėtų išlikti

```{table} Atitinkamų Stops.lt savivaldybių VINTRA esantys GTFS duomenų rinkinio failai ir eilučių skaičius juose
:name: municipalities-stops-lt-vintra-stats

| Savivaldybė                  | agency.txt | stops.txt | routes.txt | trips.txt | stop\_times.txt | calendar.txt | calendar\_dates.txt | shapes.txt |
|------------------------------|------------|-----------|------------|-----------|-----------------|--------------|---------------------|------------|
| Alytaus miesto savivaldybė   | 1          | 196       | 19         | 955       | 21483           | 31           | 200                 | 12663      |
| Druskininkų savivaldybė      | 1          | 146       | 27         | 709       | 9043            | 18           | 216                 | 9793       |
| Kauno miesto savivaldybė     | 1          | 885       | 63         | 7320      | 190006          | 302          | 1287                | 84486      |
| Klaipėdos miesto savivaldybė | 1          | 635       | 55         | 6986      | 164636          | 335          | 1272                | 24581      |
| Panevėžio miesto savivaldybė | 1          | 240       | 20         | 1694      | 28929           | 80           | 208                 | 20390      |
| Šiaulių miesto savivaldybė   | 1          | 418       | 45         | 3089      | 62861           | 121          | 92                  | 19311      |
| Vilniaus miesto savivaldybė  | 1          | 1429      | 99         | 25307     | 508844          | 2035         | 3510                | 110281     |
```